# Hybrid digital twin of a Li-ion battery


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
# from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

import plotly.io as pio
pio.renderers.default='notebook'

### 1. Load experimental data

In [ ]:
df = pd.read_csv('discharge.csv')

In [ ]:
df = df[df['Battery'] == 'B0005']
df = df[df['Temperature_measured'] > 36] #choose battery B0005
#df['Time'] =pd.to_datetime(df['Time'], unit='s')
dfb = df.groupby(['id_cycle']).max()
dfb['Cumulated_T'] = dfb['Time'].cumsum()

In [ ]:
import plotly.express as px
fig = px.scatter_matrix(dfb.drop(columns=['Time','type', 'ambient_temperature', 
                                          'time', 'Battery']), 
                                )
fig.update_traces(marker=dict(size=2,
                              color='crimson',
                              symbol='square')),
fig.update_traces(diagonal_visible=False)
fig.update_layout(
    title='Battery dataset',
    width=900,
    height=1200,
)
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                   'paper_bgcolor': 'white',}, 
                    template='plotly_white',
                    font=dict(size=7)
                    )

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=dfb['Cumulated_T']/3600, 
                         y=dfb['Capacity'],
                         mode='lines',
                         name='Capacity',
                         marker_size=3, 
                         line=dict(color='crimson', width=3)    
                        ))
fig.update_layout(
                  title="Real-world battery discharge capacity",
                  xaxis_title="Working time [hours]",
                  yaxis_title=f"Battery capacity in amp hours (Ah)"
    )
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                   'paper_bgcolor': 'white',}, 
                    template='plotly_white')

### 2. Define a physical model

Physical model according [1]. The basic equation is:<br>
<center> $L = 1 − (1 − L' )e^{-f_d}$ </center><br>

Where $L$ is the battery lifetime and $L'$ the initial battery lifetime. $f_d$ is a Linearized degradation rate per unit time and per cycle.  It can be described as:<br>
<center> $f_d = f_d(t, δ, σ, T_c)$ </center><br> 

where $t$ is charging time, δ is the cycle depth of discharge, σ is the cycle average state of charge, and $T_c$ is cell temperature. The equation for baatery capacity could be written as follows:<br>
<center> $C = C_0e^{f_d}$ </center><br>

We have empirically found that $f_d$ aproximates to:
<center> $f_d = \frac{kT_Ci}{t}$ </center><br>

where $k= $ 0.13, $i$ the cycle number and $t$ the charge time for every cycle.

- [1] *Xu, Bolun & Oudalov, Alexandre & Ulbig, Andreas & Andersson, Göran & Kirschen, D.s. (2016). Modeling of Lithium-Ion Battery Degradation for Cell Life Assessment. IEEE Transactions on Smart Grid. 99. 1-1. 10.1109/TSG.2016.2578950.* 

In [ ]:
from math import e
L = (dfb['Capacity']-dfb['Capacity'].iloc[0:1].values[0])/-dfb['Capacity'].iloc[0:1].values[0]
K = 0.13
L_1 = 1-e**(-K*dfb.index*dfb['Temperature_measured']/(dfb['Time']))
dfb['Physics Model Capacity'] = -(L_1*dfb['Capacity'].iloc[0:1].values[0]) + dfb['Capacity'].iloc[0:1].values[0]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['Physics Model Capacity'],
                         mode='lines',
                         name='Physics-based model capacity',
                         line=dict(color='navy', 
                                   width=2.5,
                                   )))

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['Capacity'],
                         mode='markers',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ),
                         name='Real-world capacity',
                         line_color='navy'))
fig.update_layout(
    title="Physical model comparison ",
    xaxis_title="Cycles",
    yaxis_title="𝐶, Capacity [Amp Hours]")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.8
))

fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')

### 3. Compare experimental data with physical model

In [ ]:
# Mean Absolute Error
M = pd.DataFrame()
S = pd.DataFrame()
def MAE(M,S):    
    return np.sum(S-M)/len(S)

print(f'Mean Absolute Error =', round(MAE(dfb['Capacity'], dfb['Physics Model Capacity']), 3))

### 4. Hybrid digital twin 

In [ ]:
#Define inputs and outputs
physics_model_capacity = dfb['Physics Model Capacity']          # input: the simulation time series
delta_observed_and_physics_capacity = dfb['Capacity'] - dfb['Physics Model Capacity']   # output: difference between measurement and simulation

physics_model_capacity_train, physics_model_capacity_test, delta_observed_and_physics_capacity_train, delta_observed_and_physics_capacity_test = train_test_split(physics_model_capacity, delta_observed_and_physics_capacity, test_size=0.33)

In [ ]:
physics_model_capacity_train.shape

In [ ]:
#The Dense function in Keras constructs a fully connected neural network layer, automatically initializing the weights as biases.
#First hidden layer
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

In [ ]:
epochs = 100
loss = "mse"
model.compile(optimizer='adam',
              loss=loss,
              metrics=['mae'], #Mean Absolute Error
             )
history = model.fit(physics_model_capacity_train, delta_observed_and_physics_capacity_train, 
                    shuffle=True, 
                    epochs=epochs,
                    batch_size=20,
                    validation_data=(physics_model_capacity_test, delta_observed_and_physics_capacity_test), 
                    verbose=1)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(0, epochs, 1),
                         y=history.history['mae'],
                         mode='lines',
                         name=f'Training Mean Absolute Error',
                         marker_size=3, 
                         line_color='orange'))
fig.add_trace(go.Scatter(x=np.arange(0, epochs, 1),
                         y=history.history['val_mae'],
                         mode='lines',
                         name=f'Validation Mean Absolute Error',
                         line_color='grey'))

fig.update_layout(
                  title="Network training",
                  xaxis_title="Epochs",
                  yaxis_title=f"Mean Absolute Error")
fig.update_layout({'plot_bgcolor': '#f2f8fd' , 
                   'paper_bgcolor': 'white',}, 
                   template='plotly_white')

### 4. Compile the hybrid digital twin

In [ ]:
physics_model_capacity_train, physics_model_capacity_test, delta_observed_and_physics_capacity_train, delta_observed_and_physics_capacity_test

fig = go.Figure()
fig.add_trace(go.Scatter(x=physics_model_capacity_train,
                         y=delta_observed_and_physics_capacity_train,
                         mode='markers',
                         name=f'Modelled Capacity',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ), 
                        line_color='crimson'))
fig.add_trace(go.Scatter(x = physics_model_capacity_train, 
                         y=model.predict(physics_model_capacity_train).reshape(-1),
                         mode='lines',
                         name=f'Trained Capacity',
                         line=dict(color='navy', width=3)))
fig.update_layout(
    title="Neural network training",
    xaxis_title="Modelled capacity",
    yaxis_title="Δ (Modelled Capacity - Measured Capacity)")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.45
))
fig.update_layout({'plot_bgcolor': '#f2f8fd' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')

In [ ]:
predicted_hybrid_twin = physics_model_capacity + model.predict(physics_model_capacity).reshape(-1)

fig = go.Figure()

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=predicted_hybrid_twin,
                         mode='lines',
                         name=f'Hybrid digital twin',
                         line=dict(color='firebrick', width=3)))

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['Physics Model Capacity'],
                         mode='lines',
                         name=f'Physics Model Capacity',
                         line=dict(color='navy', 
                                   width=3,
                                   dash='dash')))

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['Capacity'],
                         mode='markers',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ),
                         name=f'Observed capacity',
                         line_color='navy'))

fig.update_layout(
    title="Comparison of hybrid twin with other models",
    xaxis_title="Cycles",
    yaxis_title="Capacity in Amp Hours")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.77
))
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')

## 5. Prediction with hybrid twin model


In [ ]:
cycles = np.arange(168,500,1)
temperature = dfb['Temperature_measured'].iloc[167]
time = dfb['Time'].iloc[167]
K = 0.13
L_e = 1-e**(-K*cycles*temperature/time)
X_in_e = -(L_e*dfb['Capacity'].iloc[0:1].values[0]) + dfb['Capacity'].iloc[0:1].values[0]
C_twin_e = X_in_e + model.predict(X_in_e).reshape(-1)

In [ ]:
predicted_hybrid_twin = physics_model_capacity + model.predict(physics_model_capacity).reshape(-1)

fig = go.Figure()

fig.add_trace(go.Scatter(x=cycles, 
                         y=X_in_e,
                         mode='lines',
                         name=f'Physics-based model capacity (predicted)',
                         line=dict(color='navy', 
                                   width=3,
                                   dash='dash')))
fig.add_trace(go.Scatter(x=cycles, 
                         y=C_twin_e,
                         mode='lines',
                         name=f'C Digital twin (predicted)',
                         line=dict(color='crimson', 
                                   width=3,
                                   dash='dash'
                                  )))

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=X_twin,
                         mode='lines',
                         name=f'Digital twin modelled capacity',
                         line=dict(color='crimson',
                                   width=2)))
fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['Physics Model Capacity'],
                         mode='lines',
                         name=f'Physics-based model capacity',
                         line=dict(color='navy', 
                                   width=2)))

fig.update_layout(
    title="Battery capacity prediction",
    xaxis_title="Cycles",
    yaxis_title="Battery capacity [Ahr]")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.72
))
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')